In [1]:
from parlai.core.agents import create_agent
from parlai.agents.emely.emely import EmelyAgent
from parlai.core.opt import Opt
from pathlib import Path
import torch
from parlai.scripts.torchscript import export_emely
from parlai.core.agents import create_agent

# Initialize model settings

model_path = Path.cwd() / 'data/models/blender/blender_90M/'
assert model_path.is_dir()

opt_path = model_path / 'model.opt'
opt = Opt.load(opt_path)

# Change opts
opt['skip_generation'] = False
opt['init_model'] = (model_path / 'model').as_posix()
opt['no_cuda'] = True  # Cloud run doesn't offer gpu support

# Inference options
opt['inference'] = 'greedy' # 'beam'
opt['beam_size'] = 1

opt["scripted_model_file"] = "emely_scripted_test.pt"
opt["script-module"] = "parlai.torchscript.modules:TorchScriptGreedySearch"
opt["model_file"] = opt["init_model"]

opt["temp_separator"] = "__space__"

opt["bpe_add_prefix_space"] = False

opt["input"] = "Hi! What do you like to do?"

In [2]:
# Test exporting to torchscript
original_module, scripted_module = export_emely(opt)

12:04:56 | loading dictionary from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model.dict
12:04:56 | num words = 54944
12:04:56 | DEPRECATED: XLM should only be used for backwards compatibility, as it involves a less-stable layernorm operation.
12:04:57 | Total parameters: 87,508,992 (87,508,992 trainable)
12:04:57 | Loading existing model params from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model


/home/ckjellson/code/emely-models/ParlAI/parlai/agents/transformer/modules/encoder.py:183: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if positions.max().item() > self.n_positions:
/home/ckjellson/code/emely-models/ParlAI/parlai/agents/transformer/modules/attention.py:147: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert (
/home/ckjellson/anaconda3/envs/emelymodels/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'tr

torch.Size([1, 2])
2
torch.Size([1, 2])
2

Generating given the original unscripted module:
 TEXT: Hi! What do you like to do?
LABEL: __unk__

Generating given the scripted module:
 TEXT: Hi! What do you like to do?
LABEL: i love to do it ! i ' m a bit of a perfectionist .


In [3]:
# Create model to test tokenization etc
emely_agent = EmelyAgent(opt)

12:06:01 | loading dictionary from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model.dict
12:06:01 | num words = 54944
12:06:02 | Total parameters: 87,508,992 (87,508,992 trainable)
12:06:02 | Loading existing model params from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model


In [11]:
text = "Hi Emely, how are you?\nI'm good thanks! What do you do for work?\nI write code and I drink coffe."
reply = emely_agent.observe_and_act(text)
print(reply)
reply = scripted_module(text)
print(reply)

that ' s cool ! i ' ve never written code before . i ' d love to learn how to code .
that ' s cool ! i ' m a software engineer .


In [26]:
# Test performance
from time import time
nruns = 3
t1 = time()
for i in range(nruns):
    reply = emely_agent.observe_and_act("\nHi!\n My name is something and my friend's name is something likewise.")
    print(reply)
t2 = time()
orig_time = t2-t1
print("Time using original emely_agent: ", orig_time)
for i in range(nruns):
    reply = original_module('\n'.join("Hi!\n My name is something and my friend's name is something likewise."))
    print(reply)
t3 = time()
orig2_time = t3-t2
print("Time using original module: ", orig2_time)
for i in range(nruns):
    reply = scripted_module('\n'.join("Hi!\n My name is something and my friend's name is something likewise."))
    print(reply)
t4 = time()
script_time = t4-t3
print("Time using scripted model: ", script_time)

that ' s cool . what is your name ?
that ' s cool . what is your name ?
that ' s cool . what is your name ?
Time using original emely_agent:  0.6829879283905029
goodbye
goodbye
goodbye
Time using original module:  0.24225139617919922
goodbye
goodbye
goodbye
Time using scripted model:  0.2320244312286377


In [21]:
from parlai.core.message import Message
dummy_message = Message()
dummy_message['id'] = 'localHuman'
dummy_message['text'] = 'Hi'
dummy_message['episode_done'] = False
dummy_message['label_candidates'] = None
#scripted_module.observe()

AttributeError: 'RecursiveScriptModule' object has no attribute 'observe'

In [15]:
# Test ONNX
import torch.onnx
#torch.onnx.export(original_module, "Hi! What do you like to do?", "emely-testing.onnx")

In [8]:
print(type(emely_agent.model.encoder))

<class 'parlai.agents.transformer.modules.encoder.TransformerEncoder'>


In [9]:
from parlai.utils.bpe import SubwordBPEHelper
from parlai.torchscript.modules_emely import ScriptableSubwordBpeHelper
sbpe = SubwordBPEHelper(emely_agent.opt)
joint_bpe_codes = {}
for k in sbpe.bpe.bpe_codes.keys():
    joint_bpe_codes[emely_agent.opt["temp_separator"].join(k)] = sbpe.bpe.bpe_codes[k]
ssbpe = ScriptableSubwordBpeHelper(True, joint_bpe_codes, sbpe.bpe.separator, emely_agent.opt["temp_separator"])
o1 = sbpe.encode("Hi! What is your name?\nI'm Christoffer.")
o2 = ssbpe.helper_encode("Hi! What is your name?\nI'm Christoffer.")
print(o1)
print(o2)

['H@@', 'i', '!', 'W@@', 'hat', 'is', 'your', 'name', '?', '__newln__', 'I', "'", 'm', 'C@@', 'h@@', 'ri@@', 'st@@', 'offer', '.']
['H@@', 'i', '!', 'W@@', 'hat', 'is', 'your', 'name', '?', '__newln__', 'I', "'", 'm', 'C@@', 'h@@', 'ri@@', 'st@@', 'offer', '.']
